# Język Python - Wykład 6

## Wątki

https://docs.python.org/3/library/threading.html

### Tworzenie i uruchamianie wątku

In [ ]:
import threading

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            if i % 1e4 == 0:
                print(i)
            pass
                
t1 = Thread1()
t2 = Thread1()

t1.start()
t2.start()
print("Finished")

In [ ]:
p = threading.Thread(target=print, args=('bob', 'john'), kwargs={'sep': '\t'})
p.start()

In [ ]:
@threaded
def f():
    pass

f()

### Synchronizacja wątków

#### Lock

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        running = True
        while running:
            lock.acquire()
            if not queue:
                running = False
            else:
                elem = queue.pop()
            lock.release()
            time.sleep(2) # do something with elem
            lock.acquire()
            print(self.getName(), elem)
            lock.release()
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(3) # do something with elem
            with lock:
                print(self.getName(), elem)
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

`Lock`:
* może zostać zwolniony przez **dowolny wątek**
* próba ponownego zajęcia przez ten sam wątek blokuje go *ad infinitum*
* nie można zwolnić, jeżeli nie został zajęty
* można spróbować zająć w trybie nieblokującym
* nie jest automatycznie zwalniany

Polecam `RLock`:
* może zostać zwolniony tylko przez wątek, który go posiada
* można go zajmować wielokrotnie, ale należy zwolnić tyle samo razy
* jest zwalniany automatycznie po zakończeniu wątku

In [ ]:
type(threading.Lock())

In [ ]:
import threading
import time

lock = threading.RLock()

class Thread1(threading.Thread):
    def run(self, *args):
        print(lock.acquire(False))
        # lock.acquire(timeout=1)  # alt
        time.sleep(1)
        
                
Thread1().start()
# time.sleep(2)
Thread1().start()

#### Condition

In [ ]:
import threading
import time

cv = threading.Condition()

l = []

class Consumer(threading.Thread):
    def run(self):
        '''Consume one item'''
        with cv:
            while True:
                while not l:
                    cv.wait()
                print(l.pop(0))

class Producer(threading.Thread):
    def run(self):
        '''Produce one item'''
        global l
        for i in range(10):
            with cv:
                l += [i]
                cv.notify()
            time.sleep(1)

Consumer().start()
time.sleep(2)
Producer().start()

In [ ]:
print(l)

#### Inne

* `Semaphore`
* `BoundedSemaphore`
* `Event`
* `Timer`
* `Barrier`

### Wydajność

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
            pass
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
            pass
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()

finish = time.time()
print(finish - start)

## Procesy

https://docs.python.org/3/library/multiprocessing.html

### Wydajność

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
            pass
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
            pass
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()
finish = time.time()
print(finish - start)

Procesy są cięższe i mniej wygodne w użyciu, ale omijają GIL.

In [ ]:
import multiprocessing
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(multiprocessing.Process):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(2) # do something with elem
            with lock:
                print(self.pid, elem)
                        
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()

Przydatne klasy:
- Lock
- Queue
- JoinableQueue
- Pipe
- Value
- Array

## Wyrażenia regularne

https://docs.python.org/3/library/re.html

In [ ]:
"ABC123".isupper()

In [ ]:
import regex
print(regex.fullmatch('[A-Z]+', "ABC123"))
print(regex.fullmatch('[A-Z]+', "ABC"))

In [ ]:
s = '\\'
print(regex.match('\\\\', s))

In [ ]:
print(s)

In [ ]:
print('\n')
print(r'\n')

Najważniejsze funkcje:
* match / fullmatch
* search
* findall / finditer
* sub
* split

In [ ]:
match = regex.match(r'[a-z]+(.[a-z]+)+', "agh.edu.pl")
print(match)

In [ ]:
regex.match(r'[a-z]+(.[a-z]+)+', "agh")

In [ ]:
match.groups()

In [ ]:
match = regex.match(r'[a-z]+(\.[a-z]+)+', "Website: agh.edu.pl")
print(match)
match = regex.search(r'[a-z]+(\.[a-z]+)+', "Website: agh.edu.pl")
print(match)

In [ ]:
match = regex.match(r'[a-z]+(\.[a-z]+)+', "koło.pl")
print(match)
match = regex.match(r'\w+(\.\w+)+', "koło.pl")
print(match)

Wyrażenie regularne może zawierać:
* znaki
* klasy znaków: `.`, `[a-z]`, `\w`
* operatory powtórzenia: `*`, `+`, `?`, `*?`, `+?`, `??`, `{m}`, `{m,n}`, `{m,n}?`
* operator alternatywy: `|`
* początek/koniec napisu: `^`/`$`
* grupy: `(...)`
* kontekst: `(?=...)`,`(?!...)`, `(?<=...)`, `(?<!...)`
* grupy nazwane: `(?P<grupa>...)`
* grupy nieprzechwytujące: `(?:...)`
* referencje wsteczne: `(?P=grupa)`, `\1`, `\99`
* ...

In [ ]:
regex.sub(r"([a-z])", r"_\1", "abcd")

In [ ]:
regex.split("c(?=d)", "abcdefcgh")

regex.I - case insensitive

regex.M - multiline ($ może oznaczać koniec linii, a nie tylko koniec łańcucha)

regex.S - kropka może oznaczać też koniec linii

In [ ]:
regex.match('\w+', "犬いぬ")

\w - word character
\s - whitespace
\d - digit
\b - word boundary (albo backspace)

In [ ]:
r'\p{Ll}'

Klasy znaków Unicode: https://www.fileformat.info/info/unicode/category/index.htm

## Django

https://www.djangoproject.com/

Fragmenty kodu za: https://matthewdaly.co.uk/blog/2013/12/28/django-blog-tutorial-the-next-generation-part-1/

** The Web framework for perfectionists with deadlines **

* DRY - Don't Repeat Yourself. "Every piece of knowledge must have a single, unambiguous, authoritative representation within a system." http://c2.com/cgi/wiki?DontRepeatYourself
* MVC vs MVT - django często określane jest mianem MVT - Model View Template
* ORM - Object Relational Mapper (modele django)
* Routing URL oparty na wyrażeniach regularnych
* System szablonów 
* Cache'ing 
* Lokalizacja i tłumaczenia
* Automatyczny panel admina

### Aplikacje

Projekty i aplikacje łączy zależność 'wiele do wielu'.

Plik `settings.py` określa aplikacje zainstalowane w projekcie.

### ORM (Object-Relational Mapping)

Plik `models.py` w katalogu aplikacji.

In [ ]:
from django.db import models

# Create your models here.

class Post(models.Model):
    title = models.CharField(max_length=200)
    pub_date = models.DateTimeField()
    text = models.TextField()

In [ ]:
from django.db import models
[cls for cls in dir(models) if cls.endswith("Field")]

### Routing URL

In [ ]:
#plik urls.py projektu

from django.conf.urls import include, url

from django.contrib import admin

urlpatterns = [
    url(r'^admin/', include(admin.site.urls)),
    url(r'^.*$', include('blogengine.urls')),
]

In [ ]:
# plik urls.py aplikacji

from django.conf.urls import patterns, url
from django.views.generic import ListView

from blogengine.models import Post
from blogengine import views

urlpatterns = [
    url('^$', ListView.as_view(model=Post,)),
    url('^(?P<id>\d+)$', views.post_view, name='postview')
]

### Widoki

In [ ]:
# w pliku views.py aplikacji

@login_required(login_url='/user/login/')
def post_view(request, id):
    post = Post.objects.filter(id=id)[0]
    context = {'post': post}
    return render(request, 'views/post.html', context)

### Szablony

Alternatywnie: